# Setting & Load Data

In [1]:
import pandas as pd

from tqdm.notebook import tqdm
import pickle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_path = '/content/drive/MyDrive/메인프로젝트(5조)/chat_bot/data/증강결과/호그와트_qna_origin_sim_movie_qin_sy.xlsx'
val_path = '/content/drive/MyDrive/메인프로젝트(5조)/chat_bot/data/증강결과/호그와트_qna_origin_sim_movie_qin_sy.xlsx'

train_df = pd.read_excel(train_path)
val_df = pd.read_excel(val_path)

In [4]:
print(train_df.shape, val_df.shape)

(6538, 4) (6538, 4)


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6538 entries, 0 to 6537
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     6538 non-null   int64 
 1   category  6538 non-null   object
 2   question  6538 non-null   object
 3   answer    6538 non-null   object
dtypes: int64(1), object(3)
memory usage: 204.4+ KB


In [6]:
train_df.head()

,index,category,question,answer
0,1,원본,호그와트는 어떤 학교야?,만 11~18세까지의 청소년 마법사를 교육하는 7학년제 영국의 유일한 마법 학교야.
1,2,원본,호그와트는 언제 설립됐어?,"호그와트는 990년대에 고드릭 그리핀도르, 헬가 후플푸프, 로위너 래번클로, 살라자..."
2,3,원본,호그와트로 가는 방법을 알려줘.,호그와트를 가려면 킹스 크로스역 9와 4분의3 승강장에서 기차를 타고 가야해.
3,4,원본,입학허가서를 받지 않은 머글들에게 호그와트가 보일까?,허락받지 않은 머글들에게는 보이지 않아. 호그와트 앞을 지나가려고 하면 다 쓰려져가...
4,5,원본,호그와트는 안전한 장소일까?,호그와트는 기본적으로 학생의 안전을 매우 중요시하기 때문에 온갖 방어마법이나 고대 ...


In [7]:
val_df.head()

,index,category,question,answer
0,1,원본,호그와트는 어떤 학교야?,만 11~18세까지의 청소년 마법사를 교육하는 7학년제 영국의 유일한 마법 학교야.
1,2,원본,호그와트는 언제 설립됐어?,"호그와트는 990년대에 고드릭 그리핀도르, 헬가 후플푸프, 로위너 래번클로, 살라자..."
2,3,원본,호그와트로 가는 방법을 알려줘.,호그와트를 가려면 킹스 크로스역 9와 4분의3 승강장에서 기차를 타고 가야해.
3,4,원본,입학허가서를 받지 않은 머글들에게 호그와트가 보일까?,허락받지 않은 머글들에게는 보이지 않아. 호그와트 앞을 지나가려고 하면 다 쓰려져가...
4,5,원본,호그와트는 안전한 장소일까?,호그와트는 기본적으로 학생의 안전을 매우 중요시하기 때문에 온갖 방어마법이나 고대 ...


In [8]:
def make_data_set(df, purpose):

  if purpose == 'train':
    
    num = 31

  if purpose == 'val':
    
    num = 31

  df = df[['index',
           'question', 'answer']]

  data_source = []

  for idx in tqdm(range(df.shape[0])):


    dict_data_source = {
        'context':[],
        'responses':[],
        'labels': [1] + [0] * num
    }

    list_talk = list(df.iloc[idx])

    no_nan = [i for i in list_talk if pd.isnull(i) == False]

    id = no_nan[0]
    talk = no_nan[1:-1]
    answer = no_nan[-1]

    for i in range(len(talk)):

      if i % 2 == 0:

        dict_data_source['context'].append('participant 1: ' + talk[i].strip())

      else:

        dict_data_source['context'].append('participant 2: ' + talk[i].strip())  

    cond_id = (df['index']==id)

    while True:
      dict_data_source['responses'] = []
      dict_data_source['responses'].append(answer.strip())

      wrong_df = df.loc[~cond_id].sample(num)

      for wrong_idx in range(num):

        list_wrong = list(wrong_df.iloc[wrong_idx])

        wrong_no_nan = [i for i in list_wrong if pd.isnull(i) == False]

        wrong = wrong_no_nan[-1].strip()

        dict_data_source['responses'].append(wrong.strip())

      list_responses = dict_data_source['responses']

      if len(list_responses) != len(set(list_responses)):

        pass

      else:
        
        break

    data_source.append(dict_data_source)

  return data_source

# Train_Data_Source

In [9]:
print(train_df.shape)
print(train_df.columns)
print(train_df.info())

(6538, 4)
Index(['index', 'category', 'question', 'answer'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6538 entries, 0 to 6537
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     6538 non-null   int64 
 1   category  6538 non-null   object
 2   question  6538 non-null   object
 3   answer    6538 non-null   object
dtypes: int64(1), object(3)
memory usage: 204.4+ KB
None


In [10]:
train_data_source = make_data_set(train_df, purpose='train')

  0%|          | 0/6538 [00:00<?, ?it/s]

In [ ]:
with open('/content/drive/MyDrive/메인프로젝트(5조)/chat_bot/data/train_data_source_hogwart_final.pickle', 'wb') as fw:
  pickle.dump(train_data_source, fw)

In [ ]:
with open('/content/drive/MyDrive/메인프로젝트(5조)/chat_bot/data/train_data_source_hogwart_final.pickle', 'rb') as fr:
  train_data_source = pickle.load(fr)

In [11]:
train_data_source[700]

{'context': ['participant 1: 저것봐 비밀의 통로야!'],
 'responses': ['어떤 곳은 호그스미드로 간다고해. 다른 통로는 모험을 통해 찾아보면 어때?',
  '라피포스, 더클리포스, 드라코니포스를 배운다고 알고 있어',
  '대표적으로 움직이는 계단과 필요의 방이 있어',
  '사실 비밀의 방은 전설일 뿐이야! 하지만 만약 찾아보겠다면 2층 복도 끝 여자화장실로 가봐.',
  '2층으로 가봐. 그녀를 만날 수 있어.',
  '임페디멘타를 써봐 좀 늦춰줄거야',
  '호그스미드에 있어',
  '볼드모트가 호그와트를 점령하고자 죽음을 먹는 자들을 이끌고 침투한 사건이 호그와트 전투야.',
  '역대 교장들의 초상화가 걸려있어. 덤블도어는 가끔 교장 초상화에게 조언을 듣는대.',
  '마법의 계약상, 불의 잔의 결정은 취소 못합니다.선택권이 없어요.지금부터 포터 군도 네 번째 챔피언입니다.',
  '포피 폼프리 부인이야.',
  '야망과 명예를 최고로 여긴다면 야심가들의 슬리데린으로 가야겠군.',
  '어서 빨리 기숙사로 돌아가자',
  '기본적으로 머글에 대해 잘 모르는 학생이 주 대상이지만 머글 출신인 학생들도 수강은 가능해.',
  '호그스헤드에서 카드게임으로 얻었대',
  '그리핀도르의 칼은 벨라트릭스 금고에 있었어.',
  '마법 방어막을 만드는 주문이었지',
  '호그와트는 영국인과 아일랜드인만 입학 가능해. 다른 나라 학생은 어지간해선 받지 않는데, 동양인이어도 영국 국적이라면 문제 없어.',
  '호그와트 내에서는 순간이동이 불가능해서 매주 수업시간마다 중앙홀의 방어마법을 잠시 걷어내고 진행해.',
  '세베루스 스네이프 교수님은 슬리데린 사감 겸 마법약 담당 교수님이야.',
  '마법 생명체 돌보기는 수업 특성상 야외수업이 되는 경우가 많아.',
  '호그와트의 마법약 과목 교수님이시지.',
  '덜로리스 엄브리지 교수님과 아미쿠스 캐로 교수님이야.',
  '매우 거대한 거미야, 집게발이 달려있고 사람 말도 할 줄 

In [12]:
for i in range(len(train_data_source)):
  if len(train_data_source[i]['responses']) != 32:
    print(i, 'error')

# Val_Data_Source

In [13]:
print(val_df.shape)
print(val_df.columns)
print(val_df.info())

(6538, 4)
Index(['index', 'category', 'question', 'answer'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6538 entries, 0 to 6537
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     6538 non-null   int64 
 1   category  6538 non-null   object
 2   question  6538 non-null   object
 3   answer    6538 non-null   object
dtypes: int64(1), object(3)
memory usage: 204.4+ KB
None


In [14]:
val_data_source = make_data_set(val_df, purpose='val')

  0%|          | 0/6538 [00:00<?, ?it/s]

In [ ]:
with open('/content/drive/MyDrive/메인프로젝트(5조)/chat_bot/data/dev_data_source_hogwart_final.pickle', 'wb') as fw:
  pickle.dump(val_data_source, fw)

In [ ]:
with open('/content/drive/MyDrive/메인프로젝트(5조)/chat_bot/data/dev_data_source_hogwart_final.pickle', 'rb') as fr:
  val_data_source = pickle.load(fr)

In [15]:
val_data_source[0]

{'context': ['participant 1: 호그와트는 어떤 학교야?'],
 'responses': ['만 11~18세까지의 청소년 마법사를 교육하는 7학년제 영국의 유일한 마법 학교야.',
  '맥고나걸 교수님이야.',
  '호그와트의 정문에는 날개달린 멧돼지상이 있고 서쪽으로는 금지된 숲, 동쪽으로는 퀴디치 경기장, 남쪽으로는 호수, 학교 뒷편으로는 온실이 있고 호그스미드의 오두막집과 연결되는 움직이는 커다란 버드나무가 있어.',
  '비밀의 방은 2층 모우닝 머틀의 화장실 세면대를 통해서 아래로 내려가면 돼.',
  '볼드모트가 호그와트를 점령하고자 죽음을 먹는 자들을 이끌고 침투한 사건이 호그와트 전투야.',
  '저기봐! 빗자루가 있어.',
  '임페디멘타, 이부빌로, 잉고르지오 스컬러스, 리닥텀 스컬러스를 배워',
  '호그와트의 창시자는 총 4명이야',
  '호그와트의 가고일 동상을 찾아봐. 정확한 암호는 필수야.',
  '연회장으로 가겠구나',
  '어두운 회색 연기를 만들어서 상대편 마법사가 날 보기 힘들게 만들어',
  '트릴로니가 헤르미온느에게 " 네가 이 강의실에 들어왔을 때부터 느꼈지만 넌 점술이라는 고상한 학문엔 소질이 없다는걸 알았단다. 자, 봐라 나이는 어려도 영혼은 찌들렸고 영혼도 네가 집착하는 저 책처럼 말라 비틀어졌잖니. "라는 악평을 들었기 때문이지.',
  '기타 과목에서 순간이동, 연금술, 고대 마법은 6학년 이상만 들을 수 있어.',
  '그리핀도르 휴게실의 벽난로야.',
  '숲이나 온실에서 해.',
  '비밀의 방은 톰리들이 열었어',
  '입학은 호그와트의 작은 탑에 놓여 있는 오래된 검은 책과 비어 있는 은색 잉크병, 기다란 깃펜으로 결정돼.',
  '시빌 트릴로니 교수님이야.',
  '2층으로 가봐. 그녀를 만날 수 있어.',
  '숙제를 하거나 담소를 나누고 체스도 두지!',
  '호그와트 안에서 위험한 곳은 금지된 숲, 3층 출입금지구역이 있어.',
  '응 아니야 베라 베어토야',
  '나무 뿌리 

In [16]:
for i in range(len(val_data_source)):
  if len(val_data_source[i]['responses']) != 32:
    print(i, 'error')